In [17]:

import pandas as pd
import os
import aiohttp
import asyncio
from asyncio_throttle import Throttler

In [2]:
%load_ext dotenv
%dotenv keys.env

In [16]:
# create an .env file in the source directory with the following variables:
# usersKey=yourapikey
api_key = os.getenv('usersKey')
baseurl = "https://api-na.hosted.exlibrisgroup.com/almaws/v1/users"

In [12]:
headers = {
    "Authorization": f"apikey {api_key}",
    "Accept": "application/json",
    "override": "preferred_language"
}

In [5]:
# source file for user ids. only needs an 'id' column. comment out the second line if you don't need leading 0's.
users = pd.read_excel('users.xlsx', sheet_name='Sheet1')
users['id'] = ['0' + i for i in users['id'].astype(str)]

In [14]:
async def call_users(df, headers):

  throttler = Throttler(rate_limit=45)

  async with aiohttp.ClientSession() as client:

    awaitables = [
        get_users(client=client, throttler=throttler, headers=headers, id=i)
        for i in users['id']
    ]

    await asyncio.gather(*awaitables)


In [10]:
async def get_users(client, throttler, headers, id):
  try:
    resp = None
    url = baseurl + '/' + id
    async with client.get(url, headers=headers) as session:
      if session.status != 200:
        resp = await session.text() 
        session.raise_for_status()
      resp = await session.json() 
      modified_preferred_language = modify_user(resp['preferred_language'], "es", "Spanish") # modify the preferred language here
      resp['preferred_language'] = modified_preferred_language # update the preferred language in the user obj
      async with client.put(url, json=resp, headers=headers) as response: # update the user
                    if response.status != 200:
                        resp_text = await response.text()
                        print(f'Failed to update user: {response.status} for {id} {resp_text}')
                    else:
                        print('User updated successfully')
  except Exception as e:
    print(f'{e} for {id}')
  finally:
    pass


In [8]:
def modify_user(preferred_language, new_value, new_desc):
    preferred_language['value'] = new_value
    preferred_language['desc'] = new_desc
    return preferred_language

In [15]:
await call_users(users, headers)

 {'value': 'en', 'desc': 'English'} for 01234567
User updated successfully
